In [ ]:
import os, sys
from tabulate import tabulate
from tqdm import tqdm
import pandas as pd
from z3 import set_param

os.chdir("/home/yizhou7/mariposa")

from analysis.singleton_analyzer import SingletonAnalyzer
from base.factory import FACT
from base.exper import Experiment
from base.exper_analyzer import ExperAnalyzer
from debugger3 import Debugger3
from utils.system_utils import log_info
from verus_queries import *
from debugger.infomed_editor import InformedEditor

SOLVER = FACT.get_solver("z3_4_13_0")
VERI_CFG = FACT.get_config("verify")
FILTER_CFG = FACT.get_config("verus_quick")
QA = FACT.get_analyzer("10sec")


def get_debug_status(dbg: Debugger3):
    try:
        p_singleton = FACT.get_project_by_path(dbg.singleton_dir)
    except:
        return "singleton not created"

    e_singleton = FACT.try_get_exper(p_singleton, VERI_CFG, SOLVER)

    if e_singleton is None:
        return "singleton not ran"

    try:
        p_filter = FACT.get_project_by_path(dbg.singleton_filtered_dir)
    except:
        return "singleton not filtered"

    e_filter = FACT.try_get_exper(p_filter, FILTER_CFG, SOLVER)

    if e_filter is None:
        return "filtered but not ran"

    return e_filter

current_dbgs = dict()

for query in tqdm(UNSTABLE_10_SECS):
    if query in SKIP_FOR_NOW:
        continue

    dbg = Debugger3(query)
    current_dbgs[query] = dbg
    assert dbg.editor is not None


In [ ]:
table = []

for query, dbg in current_dbgs.items():
    p_singleton = FACT.get_project_by_path(dbg.singleton_dir)
    e_singleton = FACT.try_get_exper(p_singleton, VERI_CFG, SOLVER)
    ba = SingletonAnalyzer(e_singleton, QA)
    editor: InformedEditor = dbg.editor

    root_quants = editor.get_singleton_actions(skip_infeasible=False)
    root_quant_count = len(root_quants)
    tested_quant_count = len(dbg.get_edited_qnames(ba.qids))
    assert tested_quant_count <= root_quant_count
    tested_edit_count = len(ba.qids)

    p_filter = FACT.get_project_by_path(dbg.singleton_filtered_dir)
    e_filter = FACT.try_get_exper(p_filter, FILTER_CFG, SOLVER)
    fa = ExperAnalyzer(e_filter, QA)
    stabilized_eids = fa.get_stable_edit_ids()

    table.append(
        [
            query,
            root_quant_count,
            tested_quant_count,
            tested_edit_count,
            len(ba.errored_edits),
            len(ba.passed_edits),
            len(ba.timeout_edits),
            len(ba.unknown_edits),
            len(stabilized_eids),
        ]
    )

df = pd.DataFrame(
    table,
    columns=[
        "query",
        "root_quants",
        "tested_quants",
        "tested_edits",
        "edits_errored",
        "edits_passed",
        "edits_timeout",
        "edits_unknown",
        "stabilized_edits",
    ],
)

In [26]:
print(tabulate(df[df["stabilized_edits"]==0], headers='keys', tablefmt='psql'))

# q = "data/projs/vsystemsnew/base.z3/mimalloc-smt-config.35.smt2"
# dbg = current_dbgs[q]
# editor: InformedEditor = dbg.editor
# p_filter = FACT.get_project_by_path(dbg.singleton_filtered_dir)
# e_filter = FACT.try_get_exper(p_filter, FILTER_CFG, SOLVER)
# fa = ExperAnalyzer(e_filter, QA)

# actions_performed = []

# for eid in fa.qids:
#     ei = dbg.look_up_edit_with_id(eid)
#     qname, action = ei.get_singleton_edit()
#     actions_performed.append((qname, action.value))
    
# root_quants = editor.get_singleton_actions(skip_infeasible=False, skip_ignored=False)
# actions_performed = sorted(actions_performed, key=lambda x: x[0])

# print(len(root_quants))

# for qname, action in root_quants.items():
#     done = "N"
#     if qname in actions_performed:
#         done = "Y"
#     print(qname, action.value, done)


+----+----------------------------------------------------------------------+---------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+--------------------+
|    | query                                                                |   root_quants |   tested_quants |   tested_edits |   edits_errored |   edits_passed |   edits_timeout |   edits_unknown |   stabilized_edits |
|----+----------------------------------------------------------------------+---------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+--------------------|
| 20 | data/projs/flowcert/base.z3/permission.14.smt2                       |           156 |             153 |            221 |               6 |            184 |              29 |               2 |                  0 |
| 36 | data/projs/vsystemsnew/base.z3/mimalloc-smt-config.35.smt2           |            13 |              13 |     